# SmartHVAC Studio: Backend Worker (Layer 3)
This executable notebook acts as the **Coordination Engine**. It polls Firebase for new jobs, runs AI generation (Layer 4), executes EnergyPlus simulations (Layer 5), and uploads results.

---

## 1. Setup Environment

In [ ]:
# 1. Install Backend Dependencies
!pip install -r requirements.txt

# 2. Install Advisor's EnergyPlus Utility (from GitHub)
# Note: This installs the specific dev branch required for EKF hooks
!pip install -q "energy-plus-utility @ git+https://github.com/mugalan/energy-plus-utility.git@dev"

print("Dependencies installed.")

In [ ]:
import sys
import os

# Add the local folder to path so we can import 'backend'
sys.path.append(os.getcwd())

print("Current Working Directory:", os.getcwd())
print("Python Path Updated.")

## 2. Authentication

In [ ]:
# Check for Service Account Key
key_path = "serviceAccountKey.json"

if not os.path.exists(key_path):
    print("❌ ERROR: serviceAccountKey.json not found.")
    print("Please upload your Firebase Service Account JSON key to the file browser on the left.")
else:
    print("✅ Found serviceAccountKey.json")

## 3. Initialize Modules

In [ ]:
from backend.firebase_connector import FirebaseConnector
from backend.ai_generator import AIPipelines
from colab.simulation_runner import run_simulation_job
import time

# Initialize Connections
try:
    fb = FirebaseConnector(key_path)
    ai = AIPipelines() # API Key can be passed here or set in ENV
    print("Backend Modules Initialized Successfully.")
except Exception as e:
    print("Initialization Failed:", e)

## 4. Main Polling Loop

In [ ]:
def process_job(job_doc):
    job_id = job_doc.id
    data = job_doc.to_dict()
    
    print(f"\nProcessing Job: {job_id}")
    print(f"Description: {data.get('nlpInputText')}")
    
    try:
        # Handle Test Connection Jobs
        if data.get('status') == 'test_connection':
            print("[Test] Running AI Connectivity Check...")
            results = ai.test_connections()
            print(f"[Test] Results: {results}")
            fb.db.collection("jobs").document(job_id).update({
                "status": "tested",
                "testResults": results
            })
            return

        # Step 1: Mark as Running
        fb.update_status(job_id, "running")
        
        # Step 2: Generate IDF (Layer 4)
        selected_model = data.get('selectedModel', 'openai') # Default to openai if missing
        idf_content = ai.generate_idf_from_text(data.get('nlpInputText'), data.get('simulationConfig'), selected_model)
        fb.upload_string_as_file(job_id, idf_content, "model.idf", "idf")
        
        # Step 3: Run Simulation (Layer 5)
        print("[Sim] Writing IDF to disk...")
        with open("model.idf", "w") as f:
            f.write(idf_content)
            
        print("[Sim] Executing EnergyPlus via Wrapper...")
        config = data.get('simulationConfig', {})
        weather_file = config.get('weather_file', "USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw")
        
        sim_results = run_simulation_job(job_id, "model.idf", weather_file, config=config)

        # Step 4: Upload Results
        result_path = None
        if "plot" in sim_results:
            fb.upload_file(job_id, sim_results["plot"], "results")
            result_path = f"results/{job_id}/results.csv"
        
        if "csv" in sim_results:
            fb.upload_file(job_id, sim_results["csv"], "results")
            if not result_path: result_path = f"results/{job_id}/results.csv"
        
        # Step 5: Done
        fb.update_status(job_id, "done", result_path=result_path)
        print("Job Completed Successfully.")
        
    except Exception as e:
        print(f"Job Failed: {e}")
        fb.update_status(job_id, "error", error_msg=str(e))

print("Starting Polling Loop... (Stop cell to exit)")
while True:
    job = fb.get_next_queued_job()
    if job:
        process_job(job)
    else:
        # print("Waiting for jobs...", end="\r")
        time.sleep(5)